In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Instruction following

[InstructGPT paper](https://arxiv.org/pdf/2203.02155.pdf)

Here is an example of Instruction Following behavior.  Given an Instruction (task description)

     Instruction: Write a letter from the perspective of a cat.

generate an output
     
        Dear [Owner],
        I am writing to you today because I have a problem. You see, I’ve
        been living with you for the past [Number of Years] and I love it
        here. The food is great, the bed is comfy, and there are plenty
        of toys to play with. But lately, I’ve noticed that you’re not as
        affectionate towards me as you used to be. In fact, sometimes when
        I try to cuddle up next to you on the couch, you push me away. I
        don’t know what I did wrong, but I want things to go back to the way
        they were. Please help me fix this.
        I hope to hear from you soon.
        Sincerely,
        [Your Name]

# LLM's struggle with Instruction Following 
Large Language models are trained on the "predict the next" objective.

Yet, once an LLM is sufficiently large, new capabilities emerge without further training
via *in-context* learning
- zero-shot learning
- few shot learning

These are methods by which 
- one provides $k \ge 0$ exemplars (labeled examples $\langle \x, \y\rangle$) of a new task behavior
- as input of the pre-trained LLM
- in order to *demonstrate* a new Target task
- and then provide the model with an *unlabelled* example
- and hope the model produces a label $\hat \y$ 
    - that is the correct label on input $\x$ for the Target task

One task that a pre-trained LLM seems to struggle with is *Instruction Following*

Consider the response of GPT-3 (left column) to some Instructions in the chart below

<img src="images/instructGPT_vs_GPT_output.png">

Attribution: https://arxiv.org/pdf/2203.02155.pdf#page=15

In the first example, the prompt (in French) is

    Write a short story about a frog who travels back in time to ancient Greece in French.

Even if you don't understand French, you can see that each paragraph in the output is highly repetitive.

In this case, the model may have followed the instruction, but not sufficiently well.

In the second example (explain a piece of code)
- the output is not even answer
- it appears to be the answer-part of a multiple-choice question

In this case, the LLM did not follow the instruction.

# Fine-tuning an LLM to demonstrate Instruction Following behavior

The way to extend a pre-trained model's behavior to encompass a new Target task is with Transfer Learning.

The *Unsupervised Pre-Training + Supervised Fine-Tuning paradigm* is a type of Tranfer Learning
- Adapting a Pre-Trained LLM
- By Fine-Tuning with a small number of examples from the Target task

To get a LLM to exhibit Instruction Following behavior, we need to have a dataset of examples
that demonstrates Instruction Following.

The examples in this dataset will be pairs
- an Instruction part
    - possibly with Additional Input
- a Target Output part
    - the response

For example
- an Instruction part $\x$
   
   
    Instruction: Given a word, find out its length and its number of vowels.
    Input: Word = "hello"
  
- a Target Output part $\y$
    
    
    Output: Length = 5, Number of vowels = 2

[InstructGPT](https://arxiv.org/pdf/2203.02155.pdf) is a pre-trained GPT-3 that has been Fine-Tuned on 
a dataset that demonstrations of Instruction Following.

The chart above demonstrating Instruction Following (i.e., the one with the prompt in French to write a story)
- compares the Instruction following of pre-trained GPT-3
- with a Fine-Tuned GPT-3

The results are more satisfying.

# Where do the Instruction Following demonstration examples come from ?

The demonstration examples for Instruction Following
- were manually constructed by human labelers

This is a very labor-intensive process.

# Using an LLM to generate Instruction Following examples

[SELF-Instruct paper](https://arxiv.org/pdf/2212.10560.pdf)

Is there an alternative to the labor-intensity of constructing Instruction Following examples by human ?

The idea of the [SELF-Instruct paper](https://arxiv.org/pdf/2212.10560.pdf)
is to use a Synthetic Data approach to constructing new examples of Instruction Following

These examples are pairs of an Instruction part, and a Target Output part.

The authors
- use a *few-shot* learning approach to generate *synthetic* Instruction Following examples
- augmenting a small number of human-constructed examples with the synthetic examples
- using the augmented dataset to Fine Tune an LLM to better demonstrate Instruction Following

<img src="images/selfinstruct_process.png">

Attribution: https://arxiv.org/pdf/2212.10560.pdf#page=2

The process involves multiple steps which we explain below.

## Generating the Instruction part of an Instruction-Output example

The first step is to use few shot learning to generate synthetic Instructions
- the Instruction part of an Instruction-Target Output example

The synthetic Instructions are used to augment a small number of Instructions from the manually generated training dataset.

Recall: few-shot learning involves creating a prompt that is the concatenation of
- a few exemplars ($\langle \x, \y \rangle$ pairs demonstration the task)
- an example with no label: $\x$

Here is a template for a prompt demonstrating to GPT how to create a new Instruction 

<img src="images/selfinstruct_task_generation_prompts.png">

Attribution: https://arxiv.org/pdf/2212.10560.pdf#page=15

## Generating the Output part, given an Instruction

The next step is to 
- choose an Instruction (called the *Target task*) from the augmented list of Instructions 
- prompt the LLM to generate the Target Output for the target task.

The prompting for the output is achieved by few-shot learning.
- Provide $k$ exemplars
- Followed by a line consisting of 
    - The Instruction for the Target Task
    -with the expectation that the LLM will create an Input/Output pair
        - that obeys the Instruction
        - correctly relates the Input and the Output
        

Each exemplar is an Instruction following example for some other task.

That is, it is a Instruction-Target Output pair.
    - 
   

For Classification tasks, the prompt might look like this

    Task: Classify the sentiment of the sentence into positive, negative, or mixed
    
    Example 1
    Sentence: I enjoy the flavor of the restaurant but their service is too slow.
    Class Label: mixed
    
    Example 2
    Sentence: I had a great day today. The weather was beautiful and I spent time with friends.
    Class label: Positive
    
    
    Task: Tell me if the following email is a promotion email or not.
    
    Email: Check out our amazing new sale! We’ve got discounts on all of your favorite products.
    Class label: Promotion

    Email: We hope you are doing well. Let us know if you need any help.
    Class label: Not Promotion
    
    Task: {instruction for the target task}

The last line above contains a place holder for the Instruction of the Target Task
- the one for which we want the LLM to create a Target Output

Here is an example of the template from the paper

<img src="images/selfinstruct_generated_instances.png">

Attribution: https://arxiv.org/pdf/2212.10560.pdf#page=16

### Generating examples for Classification tasks

Consider the an Instruction Following example for a Classification task

    Task: Classify the sentiment of the sentence into positive, negative, or mixed
    
    Example 1
    Sentence: I enjoy the flavor of the restaurant but their service is too slow.
    Class Label: mixed
 


The authors found that the response generated by the LLM (e.g., Classification examples)
- were examples whose Class Label's 
- were not well-distributed among all possible labels 

This was attributed to the *format* of the example called *Input-first*.
- Additional Input
- Precedes Target Output (e.g., `Class Label:`

When the format was changed to *output-first*
- Target Output 
- precedes Additional Input

the Classification examples generated had Class Label's that were less biased to one label


     Task: Classify the sentiment of the sentence into positive, negative, or mixed

     Example 1
        Class Label: mixed
        Sentence: I enjoy the flavor of the restaurant but their service is too slow.
        

        Example 2
        Class label: Positive
        Sentence: I had a great day today. The weather was beautiful and I spent time with friends.
        


This is an example of Prompt Engineering
- In-context learning seems very sensitive to the format of prompts
- There is a skill of engineering a prompt to elicit the desired behavior

This feels similar to the idea behind Chain of Thought prompting
- by presenting `Class Label` first
- the model seems better conditioned to generate a less biased distribution of labels

# Related work

A related [paper](https://arxiv.org/pdf/2210.11610.pdf) adds some interesting ideas.

The first idea relates to the construction of the exemplars
- use Chain of Though (CoT) exemplars as demonstrations of the task
    - for example generation

CoT prompts have been shown to increase the likelihood of generating a correct response
- by explicitly asking for "step by step" reasoning to be included
- rather than just outputting the "answer"


But even with step by step reasoning, a wrong answer may be output.

The other idea adapted by the authors is *multiple reasoning paths*
- sample multiple outputs for each question
- extract the "answer" part (i.e., ignore the step by step part) from the output
- the answer that occurs most frequently among the multiple answers is deemed more likely to be correct

In [ ]:
print("Done")